In [1]:

!pip install -q pymupdf pdfplumber sentence-transformers faiss-cpu tiktoken accelerate bitsandbytes gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 105.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 72.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 91.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

In [2]:
import os
import re
import torch
import fitz  # PyMuPDF
import faiss
import numpy as np
import gradio as gr
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Running on: {device}")

2026-01-30 11:06:12.831563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769771173.013478      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769771173.066223      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769771173.517723      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769771173.517761      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769771173.517764      55 computation_placer.cc:177] computation placer alr

✅ Running on: cuda


In [3]:
class PDFRagEngine:
    def __init__(self, embedding_model_name="BAAI/bge-base-en-v1.5"):
        # 1. Load Embedding Model (for Retrieval)
        print("Loading Embedding Model...")
        self.embedder = SentenceTransformer(embedding_model_name, device=device)
        
        # 2. Load Generation Model (The LLM)
        # SWITCHED TO TinyLlama: No login required, runs fast on Kaggle
        print("Loading LLM (TinyLlama-1.1B)...")
        self.llm_model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
        self.tokenizer = AutoTokenizer.from_pretrained(self.llm_model_id)
        self.llm = AutoModelForCausalLM.from_pretrained(
            self.llm_model_id,
            device_map="auto",
            torch_dtype=torch.float16
        )
        
        # Internal storage
        self.chunks = []
        self.index = None
        
    def _clean_text(self, text: str) -> str:
        """Internal helper to clean extracted text."""
        text = re.sub(r'\n+', ' ', text) 
        text = re.sub(r'\s+', ' ', text) 
        return text.strip()

    def ingest_pdf(self, pdf_path: str):
        """extracts text, chunks it, and builds the vector index."""
        doc = fitz.open(pdf_path)
        raw_text = ""
        
        for page in doc:
            raw_text += page.get_text()
            
        cleaned_text = self._clean_text(raw_text)
        chunk_size = 500
        overlap = 50
        
        self.chunks = [
            cleaned_text[i : i + chunk_size] 
            for i in range(0, len(cleaned_text), chunk_size - overlap)
        ]
        print(f"✅ PDF Processed: {len(self.chunks)} chunks created.")
        
        embeddings = self.embedder.encode(
            self.chunks, 
            convert_to_numpy=True, 
            normalize_embeddings=True,
            show_progress_bar=True
        )
        
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)
        self.index.add(embeddings)
        print("✅ Vector Index Built.")

    def retrieve(self, query: str, top_k=3):
        """Finds the most relevant chunks for a query."""
        query_emb = self.embedder.encode(
            [f"Represent this question for retrieving relevant passages: {query}"], 
            convert_to_numpy=True, 
            normalize_embeddings=True
        )
        scores, indices = self.index.search(query_emb, top_k)
        results = [self.chunks[idx] for idx in indices[0]]
        return results

    def generate_answer(self, query: str):
        """Retrieves context and generates an answer using the LLM."""
        context_chunks = self.retrieve(query)
        context_text = "\n\n".join(context_chunks)
        
        # TinyLlama Chat Prompt Format
        prompt = f"""<|system|>
You are a helpful assistant. Use the provided context to answer the question. If the answer is not in the context, say "I don't know".</s>
<|user|>
Context:
{context_text}

Question:
{query}</s>
<|assistant|>
"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(device)
        
        outputs = self.llm.generate(
            **inputs, 
            max_new_tokens=200, 
            temperature=0.7,
            do_sample=True 
        )
        
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the assistant's part
        final_answer = response.split("<|assistant|>")[-1].strip()
        return final_answer

In [4]:
# Initialize the engine (this takes about 30-60 seconds to load models)
rag_engine = PDFRagEngine()

Loading Embedding Model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# Find your PDF automatically
data_dir = "/kaggle/input"
pdf_path = None

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(".pdf"):
            pdf_path = os.path.join(root, file)
            break

if pdf_path:
    print(f"📄 Found PDF: {pdf_path}")
    rag_engine.ingest_pdf(pdf_path)
else:
    print("❌ No PDF found! Please upload one to Kaggle inputs.")

📄 Found PDF: /kaggle/input/pdfone-1/DL.pdf
✅ PDF Processed: 46 chunks created.


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Vector Index Built.


In [6]:
question = "What is the main methodology proposed in this paper?"

# Calling the public method of our class
answer = rag_engine.generate_answer(question)

print("-" * 50)
print(f"❓ Question: {question}")
print(f"💡 Answer: {answer}")
print("-" * 50)

--------------------------------------------------
❓ Question: What is the main methodology proposed in this paper?
💡 Answer: The main methodology proposed in this paper is to use a transformer-based co-attention mechanism to combine the commentator's text and structured event data to align words from the commentary with actual events happening on the field. The paper is an example of multi-task learning (MTL) and is trained as a MTL system to perform three tasks simultaneously: correctness classification, emotion prediction, and sentiment analysis. The qualitative examples in Table V confirm the quantitative results.
--------------------------------------------------


In [ ]:
def process_query(message, history):
    return rag_engine.generate_answer(message)

# Simple Chat Interface
demo = gr.ChatInterface(
    fn=process_query, 
    title="📚 NotebookLM Clone (Powered by Gemma-2b)",
    description="Ask questions about your PDF. The AI reads the document and answers based *only* on the text."
)

demo.launch(share=True, debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://60c8e965f986469b52.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
